# 6 - Advanced topics: Understanding trackerdict structure

Tutorial 6 gives a good, detailed introduction to the trackerdict structure step by step.
Here is a condensed summary of functions you can use to explore the tracker dictionary.


### Steps:

<ol>
    <li> <a href='#step1'> Create a short Simulation + tracker dictionary beginning to end for 1 day </a></li>
    <li> <a href='#step2'> Explore the tracker dictionary </a></li>
    <li> <a href='#step3'> Explore Save Options </a></li>
</ol>

<a id='step 1'></a>

### 1. Create a short Simulation + tracker dictionary beginning to end for 1 day

In [1]:
import bifacial_radiance
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')

simulationName = 'Tutorial 6'
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = False
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)
# starting in v0.4.0, some torque tube parameters are passed separately as a dictionary.
tubeParams = {'diameter':diameter,
              'tubetype':tubetype,
              'material':material,
              'axisofrotation':axisofrotationTorqueTube}

# Simulation range days
startdate = '11_06'     
enddate = '11_07'

# Cell Parameters
numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

demo = bifacial_radiance.RadianceObj(simulationName, path=testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readWeatherFile(epwfile, starttime=startdate, endtime=enddate)  
cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}
mymodule = demo.makeModule(name=moduletype, torquetube=torquetube, xgap=xgap, ygap=ygap, zgap=zgap, 
                           numpanels=numpanels, cellModule=cellLevelModuleParams, tubeParams=tubeParams)

sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}  
demo.set1axis(limit_angle=limit_angle, backtrack=backtrack, gcr=mymodule.sceney / pitch, cumulativesky=cumulativesky)
demo.gendaylit1axis()
demo.makeScene1axis(module=mymodule, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()
demo.analysis1axis()

path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Loading albedo, 1 value(s), 0.213 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly  WeatherFile for the year for purposes of saving Gencumulativesky temporal weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos

Module Name: Custom_Cell-Level_Module
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Custom_Cell-Level_Module updated in module.json
Creating ~22 skyfiles. 
Created 21 skyfiles in /skies/

Making ~21 .rad files for gendaylit 1-axis workflow (this takes a minute..)
21 Radfiles created in /objects/

Making 21 octfiles in root directory.
Create

{'2021-11-06_0700': {'surf_azm': 90.0,
  'surf_tilt': 1.53,
  'theta': -1.53,
  'ghi': 9,
  'dhi': 6,
  'skyfile': 'skies\\sky2_37.5_-77.33_2021-11-06_0700.rad',
  'clearance_height': 2.2428078202112136,
  'radfile': 'objects\\1axis2021-11-06_0700__C_2.24281_rtr_10.00000_tilt_1.53000_20modsx7rows_origin0,0.rad',
  'scene': <bifacial_radiance.main.SceneObj at 0x1d585fe2df0>,
  'octfile': '1axis_2021-11-06_0700.oct',
  'AnalysisObj': <bifacial_radiance.main.AnalysisObj at 0x1d585fe2f70>,
  'Wm2Front': [6.882372,
   6.883053,
   6.883717999999999,
   6.884383,
   2.16657,
   6.892325,
   6.891981,
   6.8916,
   6.891220999999999],
  'Wm2Back': [0.8538875666666667,
   0.8415811,
   0.8281460333333333,
   0.8013850333333332,
   0.6878794,
   0.7928558666666666,
   0.8162289333333334,
   0.8250041333333332,
   0.8325529],
  'backRatio': [0.1240507656228178,
   0.1222508164884843,
   0.12028757508053828,
   0.11638931826063027,
   0.3173504892575557,
   0.11501791467349452,
   0.1184145050353

<a id='step2'></a>

### 2. Explore the tracker dictionary

You can use any of the below options to explore the tracking dictionary. Copy it into an empty cell to see their contents.

In [3]:
demo.__dict__   # Shows all keys 

trackerkeys = sorted(demo.trackerdict.keys()) # get the trackerdict keys to see a specific hour.

demo.trackerdict[trackerkeys[0]] # This prints all trackerdict content
demo.trackerdict[trackerkeys[0]]['scene']  # This just prints that scene is a Scene object
demo.trackerdict[trackerkeys[0]]['scene'].__dict__ # This shows the Scene Object contents
demo.trackerdict[trackerkeys[0]]['scene'].scenex  # Addressing one of the variables in the Scene object
demo.trackerdict[trackerkeys[0]]['scene'].sceneDict # Printing the scene dictionary saved in the Scene Object
demo.trackerdict[trackerkeys[0]]['scene'].sceneDict['tilt'] # Addressing one of the variables in the scene dictionary
demo.trackerdict[trackerkeys[0]]['scene'].scene.__dict__ # Swhoing the scene dictionary inside the Scene Object values 

# Looking at the AnalysisObj results indivudally
demo.trackerdict[trackerkeys[0]]['AnalysisObj']  # This just prints that AnalysisObj is an Analysis object
demo.trackerdict[trackerkeys[0]]['AnalysisObj'].__dict__ # This shows the Analysis Object contents
demo.trackerdict[trackerkeys[0]]['AnalysisObj'].mattype # Addressing one of the variables in the Analysis Object

# Looking at the Analysis results Accumulated for the day:
demo.Wm2Back  # this value is the addition of every individual irradiance result for each hour simulated.

#  THREE WAYS OF CALLING THE SAME THING:
# (this might be cleaned up/streamlined in following releases.
demo.trackerdict[trackerkeys[0]]['scene'].scenex
demo.trackerdict[trackerkeys[0]]['scene'].moduleDict['scenex']
demo.trackerdict[trackerkeys[0]]['scene'].scene.scenex

1.046

<a id='step3'></a>

### 3. Explore Save Options

The following lines offer ways to save your trackerdict or your demo object.

In [4]:
demo.exportTrackerDict(trackerdict = demo.trackerdict, savefile = 'results\\test_reindexTrue.csv', reindex = False)
demo.save(savefile = 'results\\demopickle.pickle')


Saved to file results\demopickle.pickle
